# Batch Normalization

<img src="https://api.ning.com/files/EXPL4V-n0-S-UQNnNq6bext-hycLoK-u6aEjnY7J2UyCWgn3eFDfbFC0T*6wIFSowUo2bxbUThjv1YqkRXddrKjFeLP8ZXqE/N2.jpg" width="400" height="50">


## Introduction

Purpose of Batch Normalization is to reduce overall __Covariant Shift__ that is a result of changing parameters from the previous layers are constantly changing. The effect of utilizing Batch Normalization is the ability to use higher learning rates and be less careful about weight initialization.

## Experiment 1: Activations Over Time

### DataSet 

MNIST dataset

### Neural Network Architecture:
1. 3 Fully Connected Hidden Layers
2. 100 Activations Per Hidden Layer
3. Each Hidden Layer uses Sigmoid 
4. Weights initialized to small Gaussian Values
5. Last Layer is connected to 10 Activation Layer and Cross Entropy

### Training
Training on 50,000 steps with 60 examples each per minibatch. 

### Experimental Observation 
Comparisons Made between Baseline [ Without Batch Norm ] and Batch Norm at Every Layer

### Graphs
1. Test Accuracy of the MNIST Network trained with and without Batch Normalization vs. the number of training steps.
2. The evolution of input distributions to a typical sigmoid over the course of training shown at 15th, 50th and 85th Percentile. 

## Experiment 2: ImageNet Classification